In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def get_gauss_prob(data, mean_hombres, mean_mujeres, cov_hombres, cov_mujeres):
    data_np = data
    likelihood_class_1 = multivariate_normal.pdf(data_np, mean_hombres, cov_hombres)
    likelihood_class_2 = multivariate_normal.pdf(data_np, mean_mujeres, cov_mujeres)
    N_class_1 = len(peso_altura_hombres)
    N_class_2 = len(peso_altura_mujeres)
    prior_1 = N_class_1/(N_class_1 + N_class_2)
    prior_2 = N_class_2/(N_class_1 + N_class_2)
    total = likelihood_class_1 * prior_1 + likelihood_class_2 * prior_2
    p_class_1 = likelihood_class_1 * prior_1/total
    p_class_2 = likelihood_class_2 * prior_2/total
    return p_class_1, p_class_2
def get_acc_gauss(data, mean_hombres, mean_mujeres, cov_hombres, cov_mujeres):
    p_class_1, p_class_2 = get_gauss_prob(data[['Peso', 'Altura']].values, mean_hombres, mean_mujeres, cov_hombres, cov_mujeres)
    return ((p_class_1>p_class_2)==(data['Genero']=='Hombre')).sum()/len(p_class_1)

N=-1
data = pd.read_csv('alturas-pesos-mils-train.csv')[:N]
test = pd.read_csv('alturas-pesos-mils-test.csv')
data_hombres = data.loc[data['Genero']=='Hombre'][['Peso','Altura']].values
data_mujeres = data.loc[data['Genero']=='Mujer'][['Peso','Altura']].values

peso_altura_hombres = data.loc[data['Genero']=='Hombre'][['Peso', 'Altura']].values
peso_altura_mujeres = data.loc[data['Genero']=='Mujer'][['Peso', 'Altura']].values
mean_hombres = peso_altura_hombres.mean(axis=0)
mean_mujeres = peso_altura_mujeres.mean(axis=0)
cov_LDA = np.cov(np.vstack([peso_altura_hombres - mean_hombres, peso_altura_mujeres - mean_mujeres]).T)
print(cov_LDA)
print(np.cov(peso_altura_hombres.T)) 
print(np.cov(peso_altura_mujeres.T))

clf = LinearDiscriminantAnalysis()
clf.fit(data, data_hombres)
LinearDiscriminantAnalysis()
print(clf.predict(data_hombres))
# f = plt.figure(figsize=(20,10))
# plt.scatter(data_hombres[:,0], data_hombres[:,1], color='b', s=2, label='Hombres')
# plt.scatter(data_mujeres[:,0], data_mujeres[:,1], color='r', s=2, label='Mujeres')
# plt.xlabel('Pesos [cms]')
# plt.ylabel('Alturas [cms]')
# plt.legend()
# plt.show()
acc_train_LDA =  get_acc_gauss(data, mean_hombres, mean_mujeres, cov_LDA, cov_LDA)
print("Accuracy Train =",acc_train_LDA)
acc_test_LDA = get_acc_gauss(test, mean_hombres, mean_mujeres, cov_LDA, cov_LDA)
print("Accuracy Test =",acc_test_LDA)

ModuleNotFoundError: No module named 'sklearn.utils'

In [1]:
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score

# Cargar datos
N = -1
data = pd.read_csv('alturas-pesos-mils-train.csv')[:N]
test = pd.read_csv('alturas-pesos-mils-test.csv')

# Separar características y etiquetas
X_train = data[['Peso', 'Altura']].values
y_train = (data['Genero'] == 'Hombre').astype(int)
X_test = test[['Peso', 'Altura']].values
y_test = (test['Genero'] == 'Hombre').astype(int)

# Crear y entrenar el modelo LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Predicciones
y_train_pred = lda.predict(X_train)
y_test_pred = lda.predict(X_test)

# Calcular accuracies
acc_train_LDA = accuracy_score(y_train, y_train_pred)
acc_test_LDA = accuracy_score(y_test, y_test_pred)

print("Accuracy Train =", acc_train_LDA)
print("Accuracy Test =", acc_test_LDA)

ModuleNotFoundError: No module named 'sklearn.utils'